In [1]:
# !wget https://huggingface.co/Helsinki-NLP/opus-mt-en-mul/resolve/main/source.spm
# !wget https://huggingface.co/Helsinki-NLP/opus-mt-en-mul/resolve/main/target.spm
# !wget https://huggingface.co/Helsinki-NLP/opus-mt-en-mul/raw/main/tokenizer_config.json
# !wget https://huggingface.co/Helsinki-NLP/opus-mt-en-mul/raw/main/vocab.json

In [1]:
# !cp source.spm ep4/
# !cp target.spm ep4/
# !cp tokenizer_config.json ep4/
# !cp vocab.json ep4/

In [2]:
# !cp  -R outputs/checkpoint-199576-epoch-4/* ep4/

In [1]:
from simpletransformers.seq2seq import Seq2SeqModel,Seq2SeqArgs

2022-05-23 05:49:07.738455: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


In [2]:
from tqdm.auto import tqdm

In [3]:
model_args = Seq2SeqArgs()

In [4]:
model_args

Seq2SeqArgs(adafactor_beta1=None, adafactor_clip_threshold=1.0, adafactor_decay_rate=-0.8, adafactor_eps=(1e-30, 0.001), adafactor_relative_step=True, adafactor_scale_parameter=True, adafactor_warmup_init=True, adam_epsilon=1e-08, best_model_dir='outputs/best_model', cache_dir='cache_dir/', config={}, cosine_schedule_num_cycles=0.5, custom_layer_parameters=[], custom_parameter_groups=[], dataloader_num_workers=0, do_lower_case=False, dynamic_quantize=False, early_stopping_consider_epochs=False, early_stopping_delta=0, early_stopping_metric='eval_loss', early_stopping_metric_minimize=True, early_stopping_patience=3, encoding=None, eval_batch_size=8, evaluate_during_training=False, evaluate_during_training_silent=True, evaluate_during_training_steps=2000, evaluate_during_training_verbose=False, evaluate_each_epoch=True, fp16=True, gradient_accumulation_steps=1, learning_rate=4e-05, local_rank=-1, logging_steps=50, loss_type=None, loss_args={}, manual_seed=None, max_grad_norm=1.0, max_seq

In [5]:
model = Seq2SeqModel(
    encoder_decoder_type="marian",
    encoder_decoder_name="ep4/",
    args=model_args,
    use_cuda=True,
)

In [8]:
dir(model)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_create_training_progress_scores',
 '_decode',
 '_get_inputs_dict',
 '_get_last_metrics',
 '_load_model_args',
 '_move_model_to_device',
 'args',
 'compute_metrics',
 'config',
 'decoder_tokenizer',
 'device',
 'encoder_tokenizer',
 'eval_model',
 'evaluate',
 'is_sweeping',
 'load_and_cache_examples',
 'model',
 'predict',
 'results',
 'save_model',
 'save_model_args',
 'train',
 'train_model']

In [6]:
model.predict(["tonphai","do"])

Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

/opt/conda/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:3524: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and the tokenizer under the `as_target_tokenizer` context manager to prepare
your targets.

Here is a short example:

model_inputs = tokenizer(src_texts, ...)
with tokenizer.as_target_tokenizer():
    labels = tokenizer(tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)


['ต้นไผ่', 'ดอร์']

In [10]:
model.predict("nokphai")

Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

['นกไผ่']

In [9]:
model.predict(["phai"])

Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

['ไพ่']

In [7]:
model.predict(["wandi"])

Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

['วันดี']

In [9]:
import pandas as pd
import os

In [10]:
path_dataset = os.path.join('..','dataset','thai2rom')

train_filepaths = os.path.join(path_dataset,'train.tsv')
dev_filepaths = os.path.join(path_dataset,'dev.tsv')
test_filepaths = os.path.join(path_dataset,'test.tsv')

train_df = pd.read_csv(train_filepaths,sep="\t")
dev_df = pd.read_csv(dev_filepaths,sep="\t")
test_df = pd.read_csv(test_filepaths,sep="\t")

In [11]:
def load_data(data_path):
    new_df=pd.DataFrame()
    new_df["target_text"]=[str(i) for i in data_path['word']]
    new_df["input_text"] =[str(i) for i in data_path['roman']]
    return new_df


In [12]:
test_df = load_data(test_df)

In [13]:
pred = model.predict(list(test_df['input_text']))

Generating outputs:   0%|          | 0/12477 [00:00<?, ?it/s]

In [14]:
ground_truth = list(test_df['target_text'])

In [15]:
def clean(s):
    if s.startswith('▁'):
        s=s.replace('▁','',1)
    return s.replace('▁',' ')

In [16]:
from jiwer import cer,wer

In [17]:
cer(ground_truth,[clean(i) for i in pred])

0.1448897506082725

In [18]:
wer(ground_truth,[clean(i) for i in pred])

0.4578006921392754

In [19]:
with open("pred-ep4.txt",'w',encoding='utf-8') as f:
    f.write('\n'.join(pred))

In [20]:
model.predict(["tonto"])

Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

['ต้นต่อ']

In [21]:
model.predict(["donald"])

Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

['โดนาลด']

In [23]:
model.predict(["facebook"])

Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

['ฟาเซบอก']

In [29]:
model.predict(["nokpop"])

Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

['นกปอบ']